# Mastering TaQL

*Of: Heb geen hekel aan TaQL*

TaQL is part of [casacore](https://github.com/casacore/casacore), a set of libraries for radio astronomy data processing.

TaQL stands for "Table Query Language", but it can be used also without tables.

## Using this notebook

This notebook is written in Jupyter, but it contains a binding to the TaQL kernel. If you highlight a code cell, you can press **Shift-Enter** to evaluate it in TaQL.

You can evaluate all the TaQL commands already present in this notebook. To understand the commands, you can try to predict the outcome of a statement before evaluating it. Also, you are encouraged to change the commands: you can enter any valid TaQL-statement in this notebook.

Navigation in Jupyter notebooks can be tricky. If you are in *command mode* (not editing a cell), lots of keyboard shortcuts are active, like `"j"` and `"k"` for scrolling, `"a"` for inserting a cell, or `"dd"` for deleting one (these should be familiar if you know `vi`). If you want to type in a cell, make sure to be in *edit mode* by checking you see a blinking cursor. 

  * To go to edit mode, press Enter or double click a cell.
  * To go back to command mode, press Esc or single click another cell.

**Exercise**: what will happen if you type `"add"` in command mode, and try it to see if you're right.

## TaQL as a calculator

Although it's not the main intended use, you can use TaQL like a regular calculator:

In [1]:
6*7

42


It can do complex numbers as well:

In [2]:
(3+4i)/(8-1i)

(0.307692307692+0.538461538462j)


Most functions you expect to work are actually there:

In [3]:
sin(pi()/2)

1.0


Also lists are supported:

In [21]:
[10,34,21,0,-3.4,8]

[ 10. ,  34. ,  21. ,   0. ,  -3.4,   8. ]


In [22]:
stddev([10,34,21,0,-3.4,8])

13.8938835464


Moreover, and here it gets interesting, TaQL supports arrays:

In [4]:
array([37,1,3,34,5,7],2,3)

[[37,  1,  3],
 [34,  5,  7]]


As you see, this yields an array of two rows and three columns.

**Note**: the command line version `taql` prints this as
```
Axis Lengths: [3, 2]  (NB: Matrix in Row/Column order)
[37, 34
 1, 5
 3, 7]
```
In this notebook, it is printed in Column/Row order, just like in python and C.

Most functions also work on arrays. To take the mean of an array:

In [24]:
mean(array([37,1,3,34,5,7],2,3))

14.5


To take the mean over one axis, use `mean`**`s`** (just like in NumPy):

In [25]:
means(array([37,1,3,34,5,7],2,3),0)

[ 35.5,   3. ,   5. ]


In [26]:
means(array([37,1,3,34,5,7],2,3),1)

[ 13.66667,  15.33333]


## Units

TaQL has basic support for units:

In [27]:
4km + 3cm

4.00003 km


In [28]:
3cm + 4km

400003.0 cm


In [29]:
1km/3h

0.333333333333 km/(h)


In [30]:
sqrt(3km)

Error in TaQL command: 'using style Python SELECT sqrt(3km)'
  Error in select expression: Erronous use of function sqrt - UnitVal::UnitVal Illegal unit dimensions for root

### Angles

Angles can be given in `h:m:d` or `d:m` format, or in radians or degrees:

In [31]:
4h56m03.5 + 4d12m43.7 + 1 deg - 0.3 rad

1.08276715834 rad


If you want the result in a different unit, append that unit to an expression:

In [33]:
(4h56m03.5 + 4d12m43.7 + 1 deg - 0.3 rad) deg

62.0379883683 deg


It also helps to know that the unit of an expression will be the same of the unit of the first component:

In [241]:
0deg + 4h56m03.5 + 4d12m43.7 + 1 deg - 0.3 rad

62.0379883683 deg


Positions on the sky or earth are commonly represented as two angles. Functions for calculations with angles are built in, for example for computing the angular distance between two angles:

In [262]:
angdist([6.60417, 52.91692] deg, [0, 90] deg) deg

37.08308 deg


### Dates

![ISO 8601 was published on 06/05/88 and most recently amended on 12/01/04.](https://imgs.xkcd.com/comics/iso_8601.png "XKCD 1179")

Literal dates can be entered directly into TaQL, for example using the above ISO standard (which was introduced after the first version of casacore).

In [128]:
1981-04-01

1981/04/01


Values can be converted to dates with the function `date()` or `datetime()`. Without arguments, this gives the current date (or date + time).

In [130]:
date(0.)

1858/11/17


As you can guess from the above, dates are internally stored as modified Julian Date.

To convert a date to a pretty-printed date, you can use `cdate()`:

In [239]:
cdate(date(0.))

17-Nov-1858


Similarly for showing times there is `ctime()`, and for showing both date and time there is `cdatetime()`.

In [240]:
date() - 1981-01-04

12807.0 d


**Exercise**: when were you 10.000 days old?

### Times

The function `time()` gives the time (current time if no arguments given) in *radians*. This makes it possible to write times in the same way as angles: 

In [108]:
time() > 12h38m

True


**Exercise**: check that `datetime() - date()` (which gives a result in days) is consistent with `time()`.

## Measures

The prefix `meas.` is for functions linking to CasaCore's *measures* library. These functions make it possible to convert measures like directions, epochs, and positions from one reference frame to another.

### Times

To do really accurate computations with times, one should use Measures. When you specify a time, it is interpreted with respect to the `UTC` frame (Coordinated Universal Time). To convert to a different frame, e.g. `TAI` (International Atomic Time), use `meas.epoch`:

In [278]:
cdatetime(meas.epoch("TAI", date()+12h, "UTC"))

['2016/01/28/12:00:36.000']


Since the default time frame is `UTC`, it may be omitted.

As you see, there is a discrepancy between `UTC` and `TAI`. This is due to leap seconds.  These leap seconds are announced only half a year before (for example, here's the [announcement](ftp://hpiers.obspm.fr/iers/bul/bulc/bulletinc.49) for 2015's leap second). This is why you should regularly update your casacore data directory, and otherwise get warnings that your times may be a second wrong.

In [16]:
meas.epoch("TAI","30-Jun-2015")-meas.epoch("UTC","30-Jun-2015")

[ 35.] s


In [17]:
meas.epoch("TAI","01-Jul-2015")-meas.epoch("UTC","01-Jul-2015")

[ 36.] s


As you see, a leap second was inserted in `UTC` between June and July 2015.

**Exercise**: calculate the number of seconds between `2015-01-01 00:00 UTC` and `2016-01-01 00:00 UTC`, and explain why the answer is *not* `31536000`.

In [25]:
(2015-01-01 00:00 - 2020-01-01 00:00) s

-157766400.0 s


Available time frames are:

`"LAST"` (Local Apparent Sidereal Time), `"LMST"` (Local Mean Sidereal Time), `"GMST1"` (Greenwhich Mean ST1), `"GAST"` (Greenwhich Apparent ST1), `"UT1"`, `"UT2"`, `"UTC"`, `"TAI"`, `"TDT"`, `"TCG"`, `"TDB"`, `"TCB"`

In [45]:
cdatetime(meas.epoch("UTC","01-Jul-2015 00:00", "UTC"))

['2015/07/01/00:00:00.000']


### Positions

Positions on Earth must be given with respect to a reference frame. Two important reference frames are `WGS84` and `ITRF`. Positions can be converted between reference frames with the function `meas.position` (or `meas.pos`).

In [179]:
meas.position("ITRF", [6.60417, 52.91692] deg, "WGS")

[ 3828485.54946,   443253.26237,  5064974.012  ] m


Since `WGS` is the default, it may be omitted.

The positions of most radio telescopes are predefined:
`"ALMA"`, `"ARECIBO"`, `"ATCA"`, `"BIMA"`, `"CLRO"`, `"DRAO"`, `"DWL"`, `"GB"`, `"GBT"`, `"GMRT"`, `"IRAM PDB"`, `"IRAM_PDB"`, `"JCMT"`, `"MOPRA"`, `"MOST"`, `"NRAO12M"`, `"NRAO_GBT"`, `"PKS"`, `"SAO SMA"`, `"SMA"`, `"VLA"`, `"VLBA"`, `"WSRT"`, `"ATF"`, `"ATA"`, `"CARMA"`, `"ACA"`, `"OSF"`, `"OVRO_MMA"`, `"EVLA"`, `"ASKAP"`, `"APEX"`, `"SMT"`, `"NRO"`, `"ASTE"`, `"LOFAR"`, `"MeerKAT"`, `"KAT-7"`, `"EVN"`, `"LWA1"`, `"PAPER_SA"`, `"PAPER_GB"`, `"e-MERLIN"`, `"MERLIN2"`, `"Effelsberg"`, `"MWA32T"`, `"AMI-LA"`

The output of meas.position defaults to be in meters from the origin. By appending `LL` to the code for the frame, you get it in lat/long.

In [271]:
meas.position("WGSLL", "WSRT") deg

[  6.60417,  52.91692] deg


**Exercise**: compute the angular distance between ALMA and MeerKAT.

### Directions

Casacore knows a lot of reference frames. Conversions between them are done with `meas.dir`:

In [187]:
meas.dir("GALACTIC", [-6h52m36.7, 34d25m56.1], "J2000")

[ 1.00291,  0.61843] rad


Since `J2000` is the default, it may be omitted.

Several directions have been predefined, like all the planets, the sun and the moon, and standard sources (`"CasA"`, `"CygA"`, `"TauA"`, `"VirA"`, `"HerA"`, `"HydA"`, `"PerA"`).

If you want to convert to a coordinate frame which is tied to the Earth, it is necessary to also specify a time and a position.

In [12]:
meas.dir("AZEL", "Jupiter", datetime(), "WSRT")

[ 0.68728, -0.45221] rad


The frame of the date and time can be given explicitly (and should be if they are not `UTC` and `WGS84`, respectively):

In [13]:
meas.dir("AZEL", "Jupiter", 2000-01-01 00:00, "TAI", 
        [3826577.110, 461022.900, 5064892.758] m, "ITRF")

[-1.57846,  0.19452] rad


Frames available for directions are:

| a | b | c |

There is a special function to see when a source will be visible on a given day:

In [155]:
meas.position("WGSLL","WSRT") deg

[  6.60417,  52.91692] deg


In [136]:
ctime(meas.riseset("SUN", date(), [6.60417, 52.91692] deg))

['07:24:19.809', '16:08:33.675']


**Exercise**: when will Cassiopeia A rise tomorrow?

In [8]:
µ

Error: unicode is not supported

In [32]:
1..3

0.0174678369304 rad


In [16]:
update [create table x inx I4 limit 366] set inx=rowid()

Update result of 366 rows


In [2]:
select cdatetime(meas.epoch("UTC",1jan16))

['2016/01/01/00:00:00.000']


In [15]:
select meas.riseset('SUN',1jan16+rowid(),'UTC',"WSRT")[0],
       meas.riseset('SUN',1jan16+rowid(),'UTC',meas.pos("WGSLL",[0.1199 ,1.57011] rad)[0],
       meas.riseset('SUN',1jan16+rowid(),'UTC',"WSRT")[0] limit 31

Error in TaQL command: using style Python select meas.riseset('SUN',1jan16+rowid(),'UTC',"WSRT")[0],
       meas.riseset('SUN',1jan16+rowid(),'UTC',meas.pos("WGSLL",[0.1199 ,1.57011] rad)[0],
       meas.riseset('SUN',1jan16+rowid(),'UTC',"WSRT")[0] limit 31
  parse error at or near position 232 'limit'

In [38]:
select cdate(d[0]), str(d[0],'TIME'), str(d[1],'TIME') 
from [
      select meas.riseset('SUN',1jan16+rowid(),'UTC',[5d0m,52d0m]) as d limit 31]

Select result of 31 rows
01-Jan-2016	07:48:51	15:37:51
02-Jan-2016	07:48:44	15:38:55
03-Jan-2016	07:48:34	15:40:01
04-Jan-2016	07:48:20	15:41:10
05-Jan-2016	07:48:02	15:42:21
06-Jan-2016	07:47:42	15:43:35
07-Jan-2016	07:47:18	15:44:52
08-Jan-2016	07:46:51	15:46:11
09-Jan-2016	07:46:20	15:47:32
10-Jan-2016	07:45:47	15:48:55
11-Jan-2016	07:45:10	15:50:20
12-Jan-2016	07:44:30	15:51:47
13-Jan-2016	07:43:47	15:53:17
14-Jan-2016	07:43:00	15:54:48
15-Jan-2016	07:42:11	15:56:20
16-Jan-2016	07:41:19	15:57:55
17-Jan-2016	07:40:23	15:59:30
18-Jan-2016	07:39:25	16:01:08
19-Jan-2016	07:38:24	16:02:46
20-Jan-2016	07:37:21	16:04:26
21-Jan-2016	07:36:14	16:06:07
22-Jan-2016	07:35:05	16:07:50
23-Jan-2016	07:33:54	16:09:33
24-Jan-2016	07:32:39	16:11:17
25-Jan-2016	07:31:23	16:13:03
26-Jan-2016	07:30:03	16:14:49
27-Jan-2016	07:28:42	16:16:36
28-Jan-2016	07:27:18	16:18:23
29-Jan-2016	07:25:52	16:20:11
30-Jan-2016	07:24:24	16:22:00
31-Jan-2016	07:22:54	16:23:50


In [1]:
select cdate(d[0]) as date, str(time(d[0]),'TIME') as up, str(time(d[1]),'TIME') as down 
from [
      select meas.riseset('SUN',1jan16+inx,'UTC',[5d0m,52d0m]) as d 
      from x
      ]

Select result of 366 rows
date       	up      	down    	
01-Jan-2016	07:48:51	15:37:51
02-Jan-2016	07:48:44	15:38:55
03-Jan-2016	07:48:34	15:40:01
04-Jan-2016	07:48:20	15:41:10
05-Jan-2016	07:48:02	15:42:21
06-Jan-2016	07:47:42	15:43:35
07-Jan-2016	07:47:18	15:44:52
08-Jan-2016	07:46:51	15:46:11
09-Jan-2016	07:46:20	15:47:32
10-Jan-2016	07:45:47	15:48:55
11-Jan-2016	07:45:10	15:50:20
12-Jan-2016	07:44:30	15:51:47
13-Jan-2016	07:43:47	15:53:17
14-Jan-2016	07:43:00	15:54:48
15-Jan-2016	07:42:11	15:56:20
16-Jan-2016	07:41:19	15:57:55
17-Jan-2016	07:40:23	15:59:30
18-Jan-2016	07:39:25	16:01:08
19-Jan-2016	07:38:24	16:02:46
20-Jan-2016	07:37:21	16:04:26
21-Jan-2016	07:36:14	16:06:07
22-Jan-2016	07:35:05	16:07:50
23-Jan-2016	07:33:54	16:09:33
24-Jan-2016	07:32:39	16:11:17
25-Jan-2016	07:31:23	16:13:03
26-Jan-2016	07:30:03	16:14:49
27-Jan-2016	07:28:42	16:16:36
28-Jan-2016	07:27:18	16:18:23
29-Jan-2016	07:25:52	16:20:11
30-Jan-2016	07:24:24	16:22:00
31-Jan-2016	07:22:54	16:23:50
01-Feb-2016	0

Or, if you're really interested in where the sun is at every hour today:

In [74]:
meas.azel("SUN", date()+[0:24]h, [6.60417, 52.91692] deg) deg

[[   6.11385  -56.30321]
 [  30.35579  -53.43495]
 [  50.68577  -47.51312]
 [  67.11134  -39.73246]
 [  80.8438   -31.0091 ]
 [  93.03819  -21.949  ]
 [ 104.5546   -12.98206]
 [ 116.02526   -4.46896]
 [ 127.92826    3.23385]
 [ 140.60507    9.74133]
 [ 154.21569   14.64144]
 [ 168.65869   17.54348]
 [-176.46092   18.17195]
 [-161.71755   16.46225]
 [-147.62672   12.58665]
 [-134.44986    6.8882 ]
 [-122.15638   -0.22263]
 [-110.49178   -8.34281]
 [ -99.04783  -17.10173]
 [ -87.27926  -26.14526]
 [ -74.45842  -35.08826]
 [ -59.59219  -43.43128]
 [ -41.44212  -50.43494]
 [ -19.15597  -55.02431]] deg



In [ ]:
update bla.MS set DATA[!FLAG]=0

In [51]:
select TIME as bla from ~/projects/tim/tim.MS limit 1

Select result of 1 row
bla            	
4871282393.01 s


In [ ]:
select hdms(DELAY_DIR), "bla" as bla from ~/projects/tim/tim.MS::FIELD

In [ ]:
select * from (select DELAY_DIR, "bla" from ~/projects/tim/tim.MS::FIELD)

In [ ]:
select UVW, ANTENNA1 as ant1, ANTENNA2 as ant2, mscal.ant1name(), mscal.ant2name() from (select from ~/projects/tim/tim.MS limit 10)

Notation of angles

Notation of dates

## Tables

In [ ]:
select mean(DATA[FLAG]) as mymean, WEIGHT from ~/projects/tim/tim.MS limit 2

Columns

limit, offset, etc

Storing the output

### Using groupby

## Structure of a Measurement Set

Example with subquery

Example with mscal

## Baseline selection syntax